In [1]:
from config import *

from easyric.caas_lite import TiffSpliter
from easyric.objects import Pix4D
from easyric.io import shp, geotiff, plot
from easyric.calculate import geo2raw, geo2tiff

import os
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import tifffile



In [3]:
p = Paths("210412", year=2021)

p4d = Pix4D(project_path=p.pix4d_project, 
            raw_img_path=p.raw_img, 
            project_name=p.project_name,
            param_folder=p.pix4d_param)

[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]


In [4]:
gcp_shp = shapefile.Reader(f"{p.root}/02_GIS/gcp.shp")

In [14]:
gcp_shp.records()[0]["id"]

5

In [15]:
gcp_2d = np.zeros((0,2))
id_list = []
for i, point in enumerate(gcp_shp.shapes()):
    id_list.append(gcp_shp.records()[i]["id"])
    gcp_2d = np.vstack([gcp_2d, np.asarray(point.points)])

In [16]:
gcp_2d

array([[ 368097.35686395, 3956068.1734115 ],
       [ 368118.01213111, 3956072.17427087],
       [ 368117.16281048, 3956053.47015926],
       [ 368112.93501424, 3956044.32447808],
       [ 368101.87478832, 3956044.76305552],
       [ 368126.63773818, 3956039.17313655],
       [ 368109.05151006, 3956033.93478165],
       [ 368129.31433944, 3956020.34916215],
       [ 368108.04195444, 3956015.87963425]])

In [17]:
id_list

[5, 11, 4, 66, 7, 13, 17, 3, 1]

In [19]:
ht = geotiff.point_query(p4d.dsm_file, gcp_2d)
ht

[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]


array([96.31834412, 96.35895538, 96.30702972, 96.37060547, 96.4148407 ,
       96.46572876, 96.39712524, 96.55265045, 96.50708771])

In [21]:
csv_str = ""

for n, lonlat, h in zip(id_list, gcp_2d, ht):
    csv_str += f"{n}, {lonlat[0]}, {lonlat[1]}, {h}\n"

In [22]:
csv_str

'5, 368097.35686394863, 3956068.1734114955, 96.31834411621094\n11, 368118.0121311137, 3956072.1742708697, 96.35895538330078\n4, 368117.16281048016, 3956053.4701592554, 96.3070297241211\n66, 368112.9350142363, 3956044.324478082, 96.37060546875\n7, 368101.87478832126, 3956044.763055521, 96.41484069824219\n13, 368126.6377381831, 3956039.173136548, 96.46572875976562\n17, 368109.05151005916, 3956033.9347816524, 96.39712524414062\n3, 368129.314339435, 3956020.3491621455, 96.55265045166016\n1, 368108.04195444356, 3956015.8796342537, 96.50708770751953\n'

In [23]:
with open(f"{p.root}/02_GIS/qgis_gcp_utm54n.csv", "w") as f:
    f.write(csv_str)